In [ ]:
import os
import pandas as pd
import re

# --- File path ---
folder_path = r"C:\Users\mark.mulehi\Desktop\Depo\Performance\MONTHLY RESULTS"

# --- Step 1: Detect all relevant files ---
files = [f for f in os.listdir(folder_path) if f.startswith("All entities_") and f.endswith(".xlsx")]

# --- Function to extract reporting date from filename ---
def extract_period(filename):
    """
    Extracts the first date in the filename.
    Example: 'All entities_28 Feb 2023 vs 28 Feb 2022.xlsx'\
             -> '2023-02-28'
    """
    match = re.search(r"(\d{1,2}) (\w+) (\d{4})", filename)
    if match:
        day, month, year = match.groups()
        try:
            period = pd.to_datetime(f"{day} {month} {year}", format="%d %b %Y")
            return period.strftime("%Y-%m-%d")   # full date for SQL
        except:
            return None
    return None

# --- Step 2: Loop through and load into pandas ---
all_data = []

for file in files:
    file_path = os.path.join(folder_path, file)
    try:
        df = pd.read_excel(file_path)

        # Add metadata columns
        df["Source_File"] = file
        df["Report_Date"] = extract_period(file)  # e.g. "2023-02-28"

        all_data.append(df)
        print(f"Loaded: {file}")
    except Exception as e:
        print(f"Error reading {file}: {e}")

# --- Step 3: Combine into one DataFrame ---
if all_data:
    final_df = pd.concat(all_data, ignore_index=True)
    print("\n✅ Data loaded successfully!")
    print("Sample data with metadata:")
    print(final_df.head())
else:
    print("⚠️ No data loaded.")


Loaded: All entities_28 Feb 2023 vs 28 Feb 2022.xlsx
Loaded: All entities_28 Feb 2025 vs 29 Feb 2024.xlsx
Loaded: All entities_30 April 2023 vs 30 April 2022.xlsx
Loaded: All entities_30 April 2025 vs 30 April 2024.xlsx
Loaded: All entities_30 June 2023 vs 30 June 2022.xlsx
Loaded: All entities_30 June 2025 vs 30 June 2024 .xlsx
Loaded: All entities_30 Nov 2022 vs 30 Nov 2021.xlsx
Loaded: All entities_30 Nov 2024 vs 30 Nov 2023.xlsx
Loaded: All entities_30 Sep 2022 vs 30 Sep 2021.xlsx
Loaded: All entities_30 Sept 2024 vs 30 Sep 2023.xlsx
Loaded: All entities_31 August 2023 vs 31 August 2022.xlsx
Loaded: All entities_31 August 2025 vs 31 August 2024 MANCO.xlsx
Loaded: All entities_31 Dec 2022 vs 31 Dec 2021.xlsx
Loaded: All entities_31 Dec 2024 vs 31 Dec 2023.xlsx
Loaded: All entities_31 Jan 2023 vs 31 Jan 2022.xlsx
Loaded: All entities_31 Jan 2025 vs 31 Jan 2024.xlsx
Loaded: All entities_31 July 2023 vs 31 July 2022.xlsx
Loaded: All entities_31 July 2025 vs 31 July 2024.xlsx
Loaded: Al

In [ ]:
import os
import pyodbc
import pandas as pd
from openpyxl import load_workbook

# -----------------------------
#  SQL CONNECTION
# -----------------------------
server = r"DESKTOP-6********\SQLExpress"
database = "Bank_Financials"

# Connect to master with autocommit enabled (for CREATE DATABASE)
conn_master = pyodbc.connect(
    f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE=master;Trusted_Connection=yes;",
    autocommit=True
)
cursor_master = conn_master.cursor()

# Create database if not exists
cursor_master.execute(f"""
IF NOT EXISTS (SELECT name FROM sys.databases WHERE name = '{database}')
BEGIN
    CREATE DATABASE {database};
END
""")
cursor_master.close()
conn_master.close()

# Connect to our working DB
conn = pyodbc.connect(
    f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;"
)
cursor = conn.cursor()

# -----------------------------
#  METRICS AND TABLES
# -----------------------------
metrics = {
    "CustomerDeposits": "B31",
    "LoansAndAdvances": "B14",
    "GrossNPLs": "B105",
    "InterestIncome": "B66",
    "InterestExpense": "B72",
    "NonInterestIncome": "B81",
    "OperatingExpenses": "B92",
    "ProfitBeforeTax": "B93",
    "ProfitAfterTax": "B98",
    "TotalAssets": "B27",
    "TotalLiabilities": "B42",
    "ShareholdersEquity": "B53"
}

# Create tables for each metric
for table in metrics.keys():
    cursor.execute(f"""
    IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='{table}' AND xtype='U')
    CREATE TABLE {table} (
        ReportDate DATE PRIMARY KEY,
        Value DECIMAL(38,2)  -- Big enough for billions
    )
    """)
conn.commit()

# -----------------------------
#  FUNCTIONS
# -----------------------------
def insert_metric(table, date_val, value):
    """Insert into SQL if ReportDate not already present"""
    if value is None:
        return
    # Convert from KSh '000 to raw value
    value = value * 1000
    cursor.execute(f"""
        IF NOT EXISTS (SELECT 1 FROM {table} WHERE ReportDate = ?)
        INSERT INTO {table} (ReportDate, Value)
        VALUES (?, ?)
    """, date_val, date_val, value)
    conn.commit()

# -----------------------------
#  PROCESS FILES
# -----------------------------
folder_path = r"C:\Users\mark.mulehi\Desktop\Depo\Performance\MONTHLY RESULTS"
files = [f for f in os.listdir(folder_path) if f.startswith("All entities_") and f.endswith(".xlsx")]

for file in files:
    file_path = os.path.join(folder_path, file)
    try:
        wb = load_workbook(file_path, data_only=True)
        if "Financial_data" not in wb.sheetnames:
            print(f"❌ Skipping {file} (no Financial_data sheet)")
            continue

        ws = wb["Financial_data"]

        # Dates from B2 (current year) and C2 (previous year)
        current_date = ws["B2"].value
        previous_date = ws["C2"].value

        if not current_date:
            print(f"⚠️ Missing current date in {file}")
            continue

        # Extract all metrics
        for table, cell in metrics.items():
            current_val = ws[cell].value
            prev_val = ws[cell.replace("B", "C")].value  # same row, column C

            # Insert current value
            insert_metric(table, current_date, current_val)

            # Insert previous year value
            if previous_date and prev_val is not None:
                insert_metric(table, previous_date, prev_val)

        print(f"✅ Processed {file}")

    except Exception as e:
        print(f"❌ Error processing {file}: {e}")

# -----------------------------
#  CLEANUP
# -----------------------------
cursor.close()
conn.close()

print("\n🎯 ETL Completed Successfully!")


✅ Processed All entities_28 Feb 2023 vs 28 Feb 2022.xlsx
✅ Processed All entities_28 Feb 2025 vs 29 Feb 2024.xlsx
✅ Processed All entities_30 April 2023 vs 30 April 2022.xlsx
✅ Processed All entities_30 April 2025 vs 30 April 2024.xlsx
✅ Processed All entities_30 June 2023 vs 30 June 2022.xlsx
✅ Processed All entities_30 June 2025 vs 30 June 2024 .xlsx
✅ Processed All entities_30 Nov 2022 vs 30 Nov 2021.xlsx
✅ Processed All entities_30 Nov 2024 vs 30 Nov 2023.xlsx
✅ Processed All entities_30 Sep 2022 vs 30 Sep 2021.xlsx
✅ Processed All entities_30 Sept 2024 vs 30 Sep 2023.xlsx
✅ Processed All entities_31 August 2023 vs 31 August 2022.xlsx
✅ Processed All entities_31 August 2025 vs 31 August 2024 MANCO.xlsx
✅ Processed All entities_31 Dec 2022 vs 31 Dec 2021.xlsx
✅ Processed All entities_31 Dec 2024 vs 31 Dec 2023.xlsx
✅ Processed All entities_31 Jan 2023 vs 31 Jan 2022.xlsx
✅ Processed All entities_31 Jan 2025 vs 31 Jan 2024.xlsx
✅ Processed All entities_31 July 2023 vs 31 July 2022.xl